In [ ]:
%load_ext autoreload
%autoreload 2
from main  import *
import os
os.environ['CUDA_VISIBLE_DEVICES']="1"

In [ ]:
opt = Config()
opt = Config()    
opt.caption_data_path = 'caption.ckpt' # 原始数据
opt.test_img = 'img/example.jpeg' # 输入图片
opt.use_gpu = False  # 是否使用GPU(没必要)
#opt.model_ckpt='caption_0914_1947' # 预训练的模型
opt.img_feature_path = 'results.ckpt'

# 数据
dataloader = get_dataloader(opt)
_data = dataloader.dataset._data
word2ix,ix2word = _data['word2ix'],_data['ix2word']

# 模型
model = CaptionModel(opt,word2ix,ix2word)
if opt.model_ckpt:
    model.load(opt.model_ckpt)
optimizer = model.get_optimizer(opt.lr1,opt.lr2)
criterion = nn.CrossEntropyLoss()
if opt.use_gpu:
    context.set_context(mode=context.GRAPH_MODE, device_target="GPU")
else:
    context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

# 前向传播
def forward_fn(imgs, captions, lengths):
    input_captions = captions[:-1]
    target_captions = captions[0]#pack_padded_sequence(captions, lengths)[0]
    score, _ = model(imgs, input_captions, lengths)
    loss = criterion(score, target_captions)
    return loss, score

# 梯度函数
grad_fn = value_and_grad(
    forward_fn, None, optimizer.parameters, has_aux=True)

# 更新，训练
def train_step(imgs, captions, lengths):
    (loss, y), grads = grad_fn(imgs, captions, lengths)
    optimizer(grads)
    return loss, y

with SummaryRecord(log_dir="./summary_dir", network=model) as summary_record:
    for epoch in range(opt.epoch):        
        loss_meter.clear()
        for ii,(imgs, (captions, lengths),indexes)  in tqdm.tqdm(enumerate(dataloader)):
            # 训练
            imgs = Tensor(imgs)
            captions = Tensor(captions)
            loss, score = train_step(imgs, captions, lengths)

            loss_meter.update(loss.item())

            # 可视化
            if (ii + 1) % opt.plot_every == 0:
                if os.path.exists(opt.debug_file):
                    ipdb.set_trace()

                summary_record.add_value('scalar', 'loss', loss_meter.eval())

                # 可视化原始图片 + 可视化人工的描述语句
                raw_img = _data['ix2id'][indexes[0]]
                img_path = opt.img_path + raw_img
                raw_img = Image.open(img_path).convert('RGB')
                raw_img = ds.vision.ToTensor()(raw_img)

                raw_caption = captions.data[:, 0]
                raw_caption = ''.join([_data['ix2word'][ii] for ii in raw_caption])
                with open('raw_caption','a') as f:
                    f.write(raw_caption)
                summary_record.add_value('image', 'raw', raw_img)

                # 可视化网络生成的描述语句
                results = model.generate(imgs.data[0])
                with open('caption','a') as f:
                    test = '</br>'.join(results)
                    f.write(test)
        save_checkpoint(model, "./" + str(epoch) + ".ckpt")

In [ ]:
opt.use_gpu

In [ ]:
optimizer = model.get_optimizer(opt.lr1)
criterion = nn.CrossEntropyLoss()


# 统计
loss_meter = Loss()


In [ ]:
opt.img_path='/home/cy/caption_data/'